# Release Presentation

In [1]:
%%capture
import sys
!{sys.executable} -m pip install google-auth-oauthlib
!{sys.executable} -m pip install google-api-python-client
!{sys.executable} -m pip install xhtml2pdf
!{sys.executable} -m pip install pdfrw

In [2]:
import requests
import pandas as pd
import re
from modules.presentations import *
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [3]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

We get the json file with the information from
https://wmwaredata.s3.us-east-2.amazonaws.com/releases.json

In [4]:
url = 'https://wmwaredata.s3.us-east-2.amazonaws.com/releases.json'
r = requests.get(url, allow_redirects=True)
open('data/releases.json', 'wb').write(r.content)

13407

In [5]:
df = pd.read_json('data/releases.json')
df = df.rename(columns={'repo_repo': 'repo_name'})

repos = []
dates = []
descriptions = []

for i in range(len(df)):
    # Repo
    repo = df['repo_name'].iloc[i] + '\n\n' + df['repo_url'].iloc[i]
    repos.append(repo)
    # Date
    d = df['release_date'].iloc[i]        
    if d is not None:
        d = d.split('T')
        date = d[0] + '\n\n' + d[1][:-1]
    else:
        date = ''
    dates.append(date)
    # Description
    content = re.sub('<.*?>', '', df['release_name'].iloc[i])
    des = wrap_by_word(content, n=20)
    descriptions.append(des)

df['Repo'] = repos
df['Date'] = dates
df['Description'] = descriptions

df = df.rename(columns={'release_tag': 'Tag'})

df = df.drop(['release_name', 'repo_name', 'repo_url', 'release_date'], axis=1)
df = df[['Repo', 'Tag', 'Date', 'Description']]
df

,Repo,Tag,Date,Description
0,aws-jmeter-test-engine\n\nhttps://github.com/k...,v0.2.1,2021-01-29\n\n12:29:04,- Instructions simplified - Bugs fixed - New s...
1,aws-jmeter-test-engine\n\nhttps://github.com/k...,v0.2.0,2021-01-22\n\n14:06:01,* UI Updates - interface supports trigger load...
2,aws-jmeter-test-engine\n\nhttps://github.com/k...,v0.1.0,2020-10-20\n\n13:01:48,- Configured AWS Cloud Formation stack to auto...
3,glasswall-desktop\n\nhttps://github.com/k8-pro...,v1.0.0,2020-12-18\n\n17:53:54,Release Notes for v1.0.0 Policy Management app...
4,glasswall-desktop\n\nhttps://github.com/k8-pro...,v0.9.0,2020-11-27\n\n12:31:01,Release Notes for v0.9.0 Added Threat Intellig...
5,glasswall-desktop\n\nhttps://github.com/k8-pro...,v0.1.7,2020-11-21\n\n13:39:04,Release Notes for v0.1.7 Policy Control & Mana...
6,glasswall-desktop\n\nhttps://github.com/k8-pro...,v0.1.6,2020-11-13\n\n20:20:58,Release Notes for v0.1.6 Added support for Par...
7,glasswall-desktop\n\nhttps://github.com/k8-pro...,v0.1.5,2020-11-06\n\n16:48:45,Release Notes for v0.1.5 Local rebuild health ...
8,glasswall-desktop\n\nhttps://github.com/k8-pro...,v0.1.4,2020-10-29\n\n16:09:16,Release Notes for v0.1.4 Processing rebuild lo...
9,glasswall-desktop\n\nhttps://github.com/k8-pro...,v0.1.3,2020-10-21\n\n13:13:12,1. Fixed Video link 2. Fixed Help Shortcut 4. ...


In [6]:
create_presentation(df, 
                    file_name='Release Presentation', 
                    table_title='Releases', 
                    step = 4, 
                    columns_widths=[100, 40, 60, 500], 
                    columns_to_merge=['Repo'])

The presentation is in this folder: https://drive.google.com/drive/u/1/folders/1dELfGV6IMMII97tTjqSXeJwPAMEjtbDX